# Digital I/O with DITask and DOTask

This notebook demonstrates digital input and output using the `DITask` and `DOTask` classes.

In [ ]:
# Verify nidaqmx is installed
try:
    import nidaqmx
except ImportError:
    raise RuntimeError(
        "nidaqmx is not installed. Install with: pip install nidaqmx"
    )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from nidaqwrapper import DITask, DOTask, list_devices

# Display connected hardware
devices = list_devices()
print("Connected NI-DAQmx devices:")
for i, dev in enumerate(devices):
    print(f"  [{i}] {dev['name']} ({dev['product_type']})")

if not devices:
    print("\nNo devices found. Connect hardware before proceeding.")

## Digital Input (On-Demand)

Read a single sample from digital input lines without hardware timing.

In [ ]:
# Edit to match your hardware (NI line specification string)
lines = "Dev1/port0/line0:3"  # 4 lines: line0, line1, line2, line3

# On-demand mode: no sample_rate parameter
di = DITask(task_name="switches")
di.add_channel("sw_lines", lines=lines)
di.start()

# Read single sample
data = di.read()

print(f"On-demand read from {lines}:")
print(f"Data: {data}")
print(f"Shape: {data.shape}")

di.clear_task()

## Digital Input (Clocked)

Acquire buffered digital input samples with hardware timing.

In [ ]:
lines = "Dev1/port0/line0:7"  # 8 lines
sample_rate = 1000  # Hz

# Clocked mode: provide sample_rate
di = DITask(task_name="fast_di", sample_rate=sample_rate)
di.add_channel("signals", lines=lines)
di.start(start_task=True)

# Read 500 samples (blocks until ready, no sleep needed)
data = di.acquire(n_samples=500)

print(f"Clocked acquisition from {lines}:")
print(f"Acquired {data.shape[0]} samples across {data.shape[1]} lines")
print(f"Data shape: {data.shape} (n_samples, n_lines)")
print(f"Mode: {di.mode}")

# Plot the first few samples for each line
if data.shape[0] > 0:
    plt.figure(figsize=(10, 4))
    n_display = min(100, data.shape[0])
    for line_idx in range(data.shape[1]):
        plt.plot(data[:n_display, line_idx], label=f"Line {line_idx}")
    plt.xlabel("Sample")
    plt.ylabel("State")
    plt.title("Digital Input (First 100 Samples)")
    plt.legend()
    plt.grid(True)
    plt.show()

di.clear_task()

## Digital Output

Write digital output values (on-demand mode).

In [ ]:
import time

lines = "Dev1/port1/line0:3"  # 4 output lines

# On-demand mode: no sample_rate
do = DOTask(task_name="leds")
do.add_channel("led_lines", lines=lines)
do.start()

# Write boolean data: all lines HIGH
do.write([True, True, True, True])
print("Set all lines HIGH")

time.sleep(1)  # Let the output run for demonstration

# Write boolean data: alternating pattern
do.write([True, False, True, False])
print("Set alternating pattern")

time.sleep(1)  # Let the output run for demonstration

# All lines LOW
do.write(np.array([False, False, False, False]))
print("Set all lines LOW")

do.clear_task()

## Port Expansion

Specifying a port without explicit lines auto-expands to all available lines on that port.

In [ ]:
# Port-only spec (no /line suffix) — will auto-expand
port_spec = "Dev1/port0"

di = DITask(task_name="port_expansion")
di.add_channel("all_port0", lines=port_spec)
di.start()

data = di.read()

print(f"Port spec '{port_spec}' expanded to {di.number_of_ch} lines")
print(f"Channel list: {di.channel_list}")
print(f"Data shape: {data.shape}")
print(f"Data: {data}")

di.clear_task()

## Clocked Digital Output

Generate a buffered digital output pattern with hardware timing.

In [ ]:
lines = "Dev1/port1/line0:3"
sample_rate = 1000  # Hz

# Generate a square wave pattern (100 samples, 4 lines)
n_samples = 100
pattern = np.zeros((n_samples, 4), dtype=bool)
pattern[::2, 0] = True  # Line 0: alternating
pattern[::4, 1] = True  # Line 1: every 4th sample
pattern[:50, 2] = True  # Line 2: first half HIGH
pattern[25:75, 3] = True  # Line 3: middle HIGH

do = DOTask(task_name="pattern_gen", sample_rate=sample_rate)
do.add_channel("pattern_lines", lines=lines)
do.start(start_task=True)

do.write_continuous(pattern)

print(f"Generated pattern on {lines}")
print(f"Pattern shape: {pattern.shape} (n_samples, n_lines)")

# Visualize the pattern
plt.figure(figsize=(10, 4))
for line_idx in range(4):
    plt.plot(pattern[:, line_idx] + line_idx * 1.2, label=f"Line {line_idx}")
plt.xlabel("Sample")
plt.ylabel("Line + Offset")
plt.title("Digital Output Pattern")
plt.legend()
plt.grid(True)
plt.show()

do.clear_task()

## Context Manager

Use `with` for both DITask and DOTask to ensure automatic cleanup.

In [ ]:
di_lines = "Dev1/port0/line0:3"
do_lines = "Dev1/port1/line0:3"

# Digital input with context manager
with DITask("ctx_di") as di:
    di.add_channel("input_ch", lines=di_lines)
    di.start()
    data = di.read()
    print(f"DI context manager read: {data}")

print("DITask auto-cleaned up.")

# Digital output with context manager
with DOTask("ctx_do") as do:
    do.add_channel("output_ch", lines=do_lines)
    do.start()
    do.write([True, False, True, False])
    print("DO context manager wrote pattern")

print("DOTask auto-cleaned up.")

## Save/Load Config

Save digital task configurations to TOML files.

In [ ]:
# Save DITask config
di = DITask("saved_di", sample_rate=2000)
di.add_channel("input_lines", lines="Dev1/port0/line0:7")

di_config_path = "di_config.toml"
di.save_config(di_config_path)
print(f"Saved DI config to {di_config_path}")
di.clear_task()

with open(di_config_path, "r") as f:
    print("\nDI Config:")
    print(f.read())

# Reload DITask
restored_di = DITask.from_config(di_config_path)
print(f"Restored DI task: {restored_di.task_name}, mode={restored_di.mode}")
restored_di.clear_task()

# Save DOTask config
do = DOTask("saved_do")
do.add_channel("output_lines", lines="Dev1/port1/line0:3")

do_config_path = "do_config.toml"
do.save_config(do_config_path)
print(f"\nSaved DO config to {do_config_path}")
do.clear_task()

with open(do_config_path, "r") as f:
    print("\nDO Config:")
    print(f.read())

# Reload DOTask
restored_do = DOTask.from_config(do_config_path)
print(f"Restored DO task: {restored_do.task_name}, mode={restored_do.mode}")
restored_do.clear_task()